<a href="https://colab.research.google.com/github/tiff03/Plant-Growth-and-Water-Usage-Data-Analysis/blob/main/trend_line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Batch Processing to Create a Trendline for Water Usage



---

Calculate slope in batches of 3 data points and save each one

plant name: length_I, length_D, length_M, length_L


tomato: 15, 30, 28, 17




In [1]:
import numpy as np
from matplotlib import pyplot as plt
import time
from time import time, ctime
from time import struct_time
import datetime
import random
import pandas as pd
import csv
from csv import writer
from google.colab import drive

tomato_stage_data = [15, 30, 28, 17]
head = ["time stamp", "event", "sensor ID", "water reserve (mm)", "volume (mm)"]

In [2]:
from sys import path_importer_cache
path_I = '/content/drive/My Drive/SUMMER 22/intern/I_data/I_simulated_2.csv'
drive.mount('/content/drive')

minute_interval = 5 #how many minutes between each sensor read
sensor_ID = "1"
water_reserve = 10 #how much water initially in the reserve
days_since_water = 2 #days since plant has been watered - set at 2 so it waters first day
irrigation_volume = 0 
IR = 0
watered = False
water_time = 9 #what time to water the plants at
all_data = []
IR_data_point = []


today = datetime.datetime(2022, 5, 19)  #first day 5/19

end_time = today

#elapse time every 5 minutes
for t in range(tomato_stage_data[0]):       #I stage
  
  watered = False

  for h in range(24):
    time_now = end_time.time() 

    if(time_now.hour % 6 == 0):
      water_reserve -= 0.25

    for m in range(60//minute_interval):

      if(not watered and (time_now.hour == water_time and time_now.minute == 0)):
        irrigation_volume = random.randint(1,3)
        water_reserve += irrigation_volume
        watered = True
        IR_data_point = [end_time.strftime("%m/%d/%Y %H:%M:%S"), "IR", None, None, irrigation_volume]
        all_data.append(IR_data_point)

      data_point = [end_time.strftime("%m/%d/%Y %H:%M:%S"), "sensor", sensor_ID, water_reserve, None]
      all_data.append(data_point)
      end_time += datetime.timedelta(minutes = minute_interval)
      
      irrigation_volume = 0
      

with open(path_I, 'w') as file:
  writer = csv.writer(file)

  writer.writerow(head)
  
  for point in all_data:
    writer.writerow(point)

Mounted at /content/drive


In [3]:
dataset = pd.read_csv(path_I)

dataset["sensor ID"] = dataset["sensor ID"].astype(object)

#raw simulated dataFrame
dataset

,time stamp,event,sensor ID,water reserve (mm),volume (mm)
0,05/19/2022 00:00:00,sensor,1.0,9.75,NaN
1,05/19/2022 00:05:00,sensor,1.0,9.75,NaN
2,05/19/2022 00:10:00,sensor,1.0,9.75,NaN
3,05/19/2022 00:15:00,sensor,1.0,9.75,NaN
4,05/19/2022 00:20:00,sensor,1.0,9.75,NaN
...,...,...,...,...,...
4330,06/02/2022 23:35:00,sensor,1.0,23.00,NaN
4331,06/02/2022 23:40:00,sensor,1.0,23.00,NaN
4332,06/02/2022 23:45:00,sensor,1.0,23.00,NaN
4333,06/02/2022 23:50:00,sensor,1.0,23.00,NaN


In [17]:
#create two different dataFrames one of the Irrigation event and the other of the sensor event

numeric_columns = list(dataset.select_dtypes(include=['float64']).columns)
def_columns = list(dataset.select_dtypes(include=['object']).columns)

print('Numerical Columns: ',numeric_columns)
print('Defining Columns: ',def_columns)

irrigation_I = dataset[dataset['event'] == 'IR']
sensor_I = dataset[dataset['event'] == 'sensor']

irrigation_I = irrigation_I.set_index(pd.Index(list(range(len(irrigation_I)))))
sensor_I = sensor_I.set_index(pd.Index(list(range(len(sensor_I)))))

print(irrigation_I)
print(sensor_I)

Numerical Columns:  ['water reserve (mm)', 'volume (mm)']
Defining Columns:  ['time stamp', 'event', 'sensor ID']
             time stamp event sensor ID  water reserve (mm)  volume (mm)
0   05/19/2022 09:00:00    IR       NaN                 NaN          2.0
1   05/20/2022 09:00:00    IR       NaN                 NaN          2.0
2   05/21/2022 09:00:00    IR       NaN                 NaN          1.0
3   05/22/2022 09:00:00    IR       NaN                 NaN          1.0
4   05/23/2022 09:00:00    IR       NaN                 NaN          3.0
5   05/24/2022 09:00:00    IR       NaN                 NaN          2.0
6   05/25/2022 09:00:00    IR       NaN                 NaN          1.0
7   05/26/2022 09:00:00    IR       NaN                 NaN          2.0
8   05/27/2022 09:00:00    IR       NaN                 NaN          2.0
9   05/28/2022 09:00:00    IR       NaN                 NaN          3.0
10  05/29/2022 09:00:00    IR       NaN                 NaN          3.0
11  05/30/

###Create cleaned dataframe object of water usage (mm) per day

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import MaxNLocator
from scipy import stats

time_x = []
usage_y = []
interval = 24
start_time = datetime.datetime(2022, 5, 19, 00, 00, 00)
first = 0
second = 0

sensor_time = sensor_I['time stamp'].tolist()
sensor_reserve = sensor_I['water reserve (mm)'].tolist()

IR_volume = irrigation_I['volume (mm)'].tolist()

IR_index = 0

for i in range(len(sensor_time)):

  sensor_time_i = datetime.datetime.strptime(sensor_I['time stamp'][i], '%m/%d/%Y %H:%M:%S')

  if(sensor_time_i == start_time):
    water_reserve = 10

  if (sensor_time_i.hour == 23 and sensor_time_i.minute == 55):
    temp = abs(sensor_reserve[i] - IR_volume[IR_index] - water_reserve)
    usage_y.append(temp)
    sensor_time_i = sensor_time_i.strftime("%m/%d/%Y")
    sensor_time_i = datetime.datetime.strptime(sensor_time_i, '%m/%d/%Y')
    time_x.append(sensor_time_i)  #append just date
    water_reserve = sensor_reserve[i] 
    IR_index += 1

  #if(sensor_time_i.hour == 00 and sensor_time_i.minute == 00 and sensor_time_i != start_time):
    #water_reserve = sensor_reserve[i]
    
  

"""
#find the start time
first = sensor_I.loc[sensor_I['time stamp'] == start_time] #dataframe object
first_index = int(sensor_I[sensor_I['time stamp']== first['time stamp'][0]].index.values)
first_water_reserve = 0
second_water_reserve = first['water reserve (mm)'][0]

for i in range(first_index, len(sensor_I), interval * 60 // 5):
  if(first_water_reserve != 0):
    second_water_reserve = sensor_I['water reserve (mm)'][i]
    IR = irrigation_I['volume (mm)'][IR_index]
    value = abs(second_water_reserve - first_water_reserve - IR)
    usage_y.append(value)
    #time_x.append(datetime.datetime.strptime(sensor_I['time stamp'][i], '%m/%d/%Y %M:%S:%f'))
    IR_index+=1

  first_water_reserve = second_water_reserve

"""

days = [x for x in range(1, len(time_x) + 1)]

d = {'time stamp': time_x, 'days': days, 'water usage per day (mm)': usage_y}

clean_dataset = pd.DataFrame(d)

clean_dataset.to_csv('I_clean_dataset_2.csv')  

clean_dataset = clean_dataset.set_index(pd.Index(list(range(1, len(clean_dataset)+1)))) #index is now days since start day

clean_dataset

,time stamp,days,water usage per day (mm)
1,2022-05-19,1,1.0
2,2022-05-20,2,1.0
3,2022-05-21,3,1.0
4,2022-05-22,4,1.0
5,2022-05-23,5,1.0
6,2022-05-24,6,1.0
7,2022-05-25,7,1.0
8,2022-05-26,8,1.0
9,2022-05-27,9,1.0
10,2022-05-28,10,1.0


###Stat calculations


---

create new dataframe with all the stat calculations every 3 days

In [30]:
from scipy.stats import linregress

#index, start day, end day, slope

x_vals = clean_dataset['days'].tolist()
y_vals = clean_dataset['water usage per day (mm)'].tolist()
dates = clean_dataset['time stamp'].tolist()

new_head = ['start day', 'end day', 'slope (3 day)', 'slope (all points)']

start_index = 0 #different for every stage
index = start_index
batch_interval = 2  #how many away from start point
dataframe_length = len(clean_dataset)
stats_df = pd.DataFrame(columns = new_head)

while(index + batch_interval < dataframe_length):
  index_2 = index + batch_interval

  temp_x = [x_vals[index], x_vals[index_2]]
  temp_y = [y_vals[index], y_vals[index_2]]
  slope, intercept, r_value, p_value, stderr = linregress(temp_x, temp_y) #slope of regression line (3 day)

  final_x = [x_vals[start_index], x_vals[index_2]]
  final_y = [y_vals[start_index], y_vals[index_2]]
  f_slope, f_intercept, f_r_value, f_p_value, f_stderr = linregress(final_x, final_y) #slope of regression line

  #appending data
  stats_df = stats_df.append({'start day' : dates[index], 'end day' : dates[index_2], 'slope (3 day)' : slope, 'slope (all points)' : f_slope}, ignore_index = True)

  index += 1


stats_df = stats_df.set_index(pd.Index(list(range(1, len(stats_df)+1))))
stats_df


,start day,end day,slope (3 day),slope (all points)
1,2022-05-19,2022-05-21,0.0,0.0
2,2022-05-20,2022-05-22,0.0,0.0
3,2022-05-21,2022-05-23,0.0,0.0
4,2022-05-22,2022-05-24,0.0,0.0
5,2022-05-23,2022-05-25,0.0,0.0
6,2022-05-24,2022-05-26,0.0,0.0
7,2022-05-25,2022-05-27,0.0,0.0
8,2022-05-26,2022-05-28,0.0,0.0
9,2022-05-27,2022-05-29,0.0,0.0
10,2022-05-28,2022-05-30,0.0,0.0
